In [ ]:
from osgeo import gdal
from osgeo import ogr
gdal.UseExceptions()

In [ ]:
bathy = gdal.Open('data/navd_bath_30m')
assert bathy.GetDriver().GetDescription() == 'AIG' # ArcInfo Binary Grid

In [ ]:
enc = ogr.Open('data/ENC_ROOT/US4MA13M/US4MA13M.000')
assert enc.GetDriver().GetDescription() == 'S57' # S57

In [ ]:
enc.GetLayerByName('M_COVR')